In [362]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [365]:
#LOAD DATA
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
abalone = fetch_ucirepo(id=1) 
  
# data (as pandas dataframes) 
X = abalone.data.features 
y = abalone.data.targets 


In [368]:
#CLEAN DATA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import scipy
from scipy.stats import pearsonr
import sklearn
from sklearn.preprocessing import StandardScaler 
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score

all_df = pd.read_csv("./abalone.data",index_col=False)

column_titles = ['Sex', 'Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight', 'Rings']

# Assign the column titles to the DataFrame using the columns attribute.
all_df.columns = column_titles

X = all_df
X['Sex'].replace('M', 0, inplace = True)
X['Sex'].replace('F', 1, inplace = True)
X['Sex'].replace('I', 2, inplace = True)

X = all_df.drop('Rings', axis=1)

y = all_df['Rings']

In [369]:
#SPLIT DATA
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
Xs_train, Xs_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3,random_state=1)

In [389]:
#APPLY NEURAL NETWORK, MLP REGRESSOR
from sklearn.neural_network import MLPRegressor

# Create and train an MLPRegressor
mlp_regressor = MLPRegressor(random_state=1, max_iter=500).fit(Xs_train, y_train)

# Make predictions on the test data
y_pred = mlp_regressor.predict(Xs_test)

classifier_score = mlp_regressor.score(Xs_test, y_test)

print('The classifier accuracy score of MLP is {:03.2f}'.format(classifier_score))


The classifier accuracy score of MLP is 0.60


In [391]:
#K-FOLD
param_grid = {
    'hidden_layer_sizes': [2, 3],
    'max_iter': [600, 700]
}
# kf = KFold(n_splits=10, shuffle=True, random_state=1)
inner_cv = KFold(n_splits=4, shuffle=True, random_state=1)
outer_cv = KFold(n_splits=4, shuffle=True, random_state=1)
grid_search = GridSearchCV(MLPRegressor(), param_grid, cv=inner_cv, scoring='neg_mean_squared_error')

In [392]:
grid_search.fit(Xs_train, y_train)
non_nested_scores = grid_search.best_score_
print(non_nested_scores)


/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anacond

-4.974429830314619


In [380]:
nested_cv_results = cross_val_score(grid_search, Xs_train, y_train, cv=outer_cv).mean()
print(nested_cv_results)

/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anacond

-8.817689683491466


/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [354]:
best_params = grid_search.best_params_
best_mlp = grid_search.best_estimator_
print(best_params)
print(best_mlp)

{'hidden_layer_sizes': 3, 'max_iter': 1000}
MLPRegressor(hidden_layer_sizes=3, max_iter=1000)


In [317]:
#CHANGE TO LOGISTIC REGRESSION FOR REGRESSION

In [393]:
#APPLY LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=100)
log_reg.fit(Xs_train, y_train)

classifier_score_reg = log_reg.score(Xs_test, y_test)
print('The classifier accuracy score is {:03.2f}'.format(classifier_score_reg))

The classifier accuracy score is 0.27


/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [395]:
#K-FOLD
param_grid = {
    'intercept_scaling': [1, 2, 3],
    'C': [1, 2, 3],
    'max_iter': [200, 300]
}
# kf = KFold(n_splits=10, shuffle=True, random_state=1)
inner_cv = KFold(n_splits=4, shuffle=True, random_state=1)
outer_cv = KFold(n_splits=4, shuffle=True, random_state=1)
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=inner_cv, scoring='neg_mean_squared_error')

In [ ]:
grid_search.fit(Xs_train, y_train)
non_nested_scores = grid_search.best_score_
print(non_nested_scores)

/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

In [344]:
nested_cv_results = cross_val_score(grid_search, Xs_train, y_train, cv=outer_cv).mean()
print(nested_cv_results)

/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (700) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (700) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (700) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/cleon/opt/anacond

-4.935337426605136


In [ ]:
best_params = grid_search.best_params_
best_mlp = grid_search.best_estimator_
print(best_params)
print(best_mlp)